In [2]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=cb98d77ae6688e50e7ed7193bac9191cb25d83db7ac4e7d94b81489ae6eb634f
  Stored in directory: /Users/ollie/Library/Caches/pip/wheels/aa/8a/36/59ed4f6fbcb6100967618eeb0696046bf9777a41ac2ff1f9b9
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8367 sha256=5dfdf607c80d94e5f48b0b0cb8bdf6432b29bfaa0123e3d6eb6e30f2d52af10d
  Stored in directory: /Users/ollie/Library/Caches/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking


In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [7]:
options_chain("GME")

,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,mark
0,GME210312C00005000,5.0,256.50,261.65,3.0,9,56.625001,True,2021-03-13,0.000000,True,259.075
1,GME210312C00010000,10.0,252.65,256.65,5.0,0,25.906252,True,2021-03-13,0.000000,True,254.650
2,GME210312C00015000,15.0,247.65,251.65,1.0,3,22.375003,True,2021-03-13,0.000000,True,249.650
3,GME210312C00020000,20.0,241.50,246.65,54.0,0,30.835938,True,2021-03-13,0.000000,True,244.075
4,GME210312C00025000,25.0,237.65,241.65,64.0,1,18.171879,True,2021-03-13,0.000000,True,239.650
...,...,...,...,...,...,...,...,...,...,...,...,...
2819,GME230120P00570000,570.0,434.50,453.50,1.0,29,1.404605,True,2023-01-21,1.860274,False,444.000
2820,GME230120P00600000,600.0,514.00,532.00,NaN,1,1.881745,True,2023-01-21,1.860274,False,523.000
2821,GME230120P00850000,850.0,684.50,703.50,1.0,3,1.331501,True,2023-01-21,1.860274,False,694.000
2822,GME230120P00900000,900.0,802.00,820.00,NaN,1,1.899521,True,2023-01-21,1.860274,False,811.000
